In [106]:
import random
import numpy as np
import glob
import os
from pydicom import dcmread
import shutil

In [107]:
# selected top-level folder containing all the patient folders
tl_dir = '/Users/darylwilding-mcbride/Downloads/Anonymiser/test patients'

In [108]:
base_dir = os.path.dirname(tl_dir)

In [109]:
tl_folder_name = os.path.basename(tl_dir)
print(tl_folder_name)
anon_tl_folder_name = tl_folder_name + '_ANON'

test patients


In [110]:
# directories under the top-level folder
tl_files_l = glob.glob('{}/*'.format(tl_dir), recursive=False)

In [111]:
# need to replace two things in the DICOM's full path name:
# 1. the top-level folder name
# 2. the patient folder name

In [112]:
subfolder_idx = 1
for tl_file in tl_files_l:
    if os.path.isdir(tl_file):
        # this is a patient folder
        patient_folder_name = os.path.basename(tl_file)
        folder_sub = tl_dir + os.sep + patient_folder_name
        # the anon patient folder
        anon_patient_folder_name = 'Brain-{:03d}'.format(subfolder_idx)
        # to anonymise the folder names, we now have the substring to replace with
        anon_folder_sub = anon_tl_folder_name + os.sep + anon_patient_folder_name
        anon_tl_file = tl_file.replace(tl_file, anon_folder_sub)
        anon_tl_path = base_dir + os.sep + anon_tl_file
        print('---------------')
        print('{} --> {}'.format(tl_file, anon_tl_path))
        subfolder_idx += 1
        # now find the DICOMs under the patient folder
        dicom_files = glob.glob('{}/**/*.dcm'.format(tl_file), recursive=True)
        invalid_file_count = 0
        for dicom_fn in dicom_files:
            anon_dicom_fn = base_dir + os.sep + dicom_fn.replace(folder_sub, anon_folder_sub)
            anon_dicom_fn_dirname = os.path.dirname(anon_dicom_fn)
            print('copying {} --> {}'.format(dicom_fn, anon_dicom_fn))
            # create the anon folder if it doesn't exist
            if not os.path.exists(anon_dicom_fn_dirname):
                os.makedirs(anon_dicom_fn_dirname)
            # load and process the file
            try:
                ds = dcmread(dicom_fn)
            except Exception as e:
                print(e)
                invalid_file_count += 1
            else:
                ds.save_as(anon_dicom_fn)


---------------
/Users/darylwilding-mcbride/Downloads/Anonymiser/test patients/Sally Smith 12-03-2021 --> /Users/darylwilding-mcbride/Downloads/Anonymiser/test patients_ANON/Brain-001
copying /Users/darylwilding-mcbride/Downloads/Anonymiser/test patients/Sally Smith 12-03-2021/GBM_MRI/MR.1.3.6.1.4.1.9590.100.1.2.346477901911620838710846051021293417483.dcm --> /Users/darylwilding-mcbride/Downloads/Anonymiser/test patients_ANON/Brain-001/GBM_MRI/MR.1.3.6.1.4.1.9590.100.1.2.346477901911620838710846051021293417483.dcm
copying /Users/darylwilding-mcbride/Downloads/Anonymiser/test patients/Sally Smith 12-03-2021/GBM_MRI/MR.1.3.6.1.4.1.9590.100.1.2.192987874612824131101662808741288453276.dcm --> /Users/darylwilding-mcbride/Downloads/Anonymiser/test patients_ANON/Brain-001/GBM_MRI/MR.1.3.6.1.4.1.9590.100.1.2.192987874612824131101662808741288453276.dcm
copying /Users/darylwilding-mcbride/Downloads/Anonymiser/test patients/Sally Smith 12-03-2021/GBM_MRI/MR.1.3.6.1.4.1.9590.100.1.2.33764912211210